# Scripts utilisés ou pris comme référence pour le projet Movies

## 1 SQL, remplir les tables

Pour remplir toutes les tables de la base de données MySQL avec les informations des différents fichiers .tsv

In [ ]:
SET UNIQUE_CHECKS=0;
SET FOREIGN_KEY_CHECKS=0;

truncate TABLE `name_basics`;
LOAD DATA LOCAL INFILE 'name.basics.tsv'
INTO TABLE `name_basics`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

truncate TABLE `title_basics`;
LOAD DATA LOCAL INFILE 'title.basics.tsv'
INTO TABLE `title_basics`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

truncate TABLE `title_principals`;
LOAD DATA LOCAL INFILE 'title.principals.tsv'
INTO TABLE `title_principals`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

truncate TABLE `title_akas`;
LOAD DATA LOCAL INFILE 'title.akas.tsv'
INTO TABLE `title_akas`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

truncate TABLE `title_episode`;
LOAD DATA LOCAL INFILE 'title.episode.tsv'
INTO TABLE `title_episode`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

truncate TABLE `title_ratings`;
LOAD DATA LOCAL INFILE 'title.ratings.tsv'
INTO TABLE `title_ratings`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

truncate TABLE `title_crew`;
LOAD DATA LOCAL INFILE 'title.crew.tsv'
INTO TABLE `title_crew`
FIELDS TERMINATED BY '\t' ENCLOSED BY ''
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

SET UNIQUE_CHECKS=1;
SET FOREIGN_KEY_CHECKS=1;

SELECT * FROM `name_basics` 
LIMIT 5;

## 2 python, longueur maximale
Code Python pour trouver la longueur maximale des champs des différentes tables (je l'ai exécuté en spider, ayant les grandes tables sur mon ordinateur)

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

#imports
import pandas as pd

def analyse(fn, list):
    maximums={}
    N=1000000
    print("*****************************")
    print("Analyse de {} par bloc de {} lignes".format(fn, N))
    print("Attributs: {}".format(list))
    dfi = pd.read_csv(fn, chunksize=N, sep='\t')
    for df in dfi: # On boucle sur les DF d'itération
        #~ print(df)
        for attribute in list: # Boucle sur la liste 'list'
            if attribute not in maximums: maximums[attribute] = 0 # Initialisation de la variable maximums, si besoin
            #~ print("len of "+attribute)
            df[attribute+'Length'] = df[attribute].astype(str).map(len) # Calcul des lengueur de chaine de la colonne 
            maxValue = df[attribute+'Length'].max() # Récupère le max de la colonne
            maximums[attribute] = max(maxValue, maximums[attribute]) # MAJ du max dans maximums
            df = df.sort_values(by=[attribute+'Length'], ascending=False, ignore_index=False) # on trie le DF par colonne de longueur
            #~ print(df.head(1))
            #~ print("max de {}: {}".format(attribute, maxValue))
        print("maximums={}".format(maximums))
        pass


#analyse("/datalab2020/movies/title.crew.tsv", ('tconst', 'directors', 'writers'))
#analyse("/datalab2020/movies/title.akas.tsv", ('titleId', 'title', 'region', 'language', 'types', 'attributes'))
#analyse("/datalab2020/movies/title.akas-mod.tsv", ('titleId', 'title', 'region', 'language', 'types', 'attributes'))

#analyse("/datalab2020/movies/name.basics.tsv", ('nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'))
#analyse("/datalab2020/movies/title.basics.tsv", ('tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'))

#analyse("/datalab2020/movies/title.episode.tsv", ('tconst', 'parentTconst', 'seasonNumber', 'episodeNumber'))
#analyse("/datalab2020/movies/title.principals.tsv", ('tconst', 'ordering', 'nconst', 'category', 'job', 'characters'))

analyse("/datalab2020/movies/title.ratings.tsv", ('tconst', 'averageRating', 'numVotes'))



le résultat de cette recherche était:

Analyse de title.crew.tsv
maximums={'tconst': 10, 'directors': 4697, 'writers': 12458}

Analyse de title.akas.tsv
maximums={'titleId': 10, 'title': 56276, 'region': 4, 'language': 3, 'types': 20, 'attributes': 62}

**title.akas-mod.tsv
maximums={'titleId': 10, 'title': 831, 'region': 4, 'language': 3, 'types': 20, 'attributes': 62}

name.basics.tsv
maximums={'nconst': 10, 'primaryName': 105, 'birthYear': 4, 'deathYear': 4, 'primaryProfession': 66, 'knownForTitles': 61}

title.basics.tsv
maximums={'tconst': 10, 'titleType': 12, 'primaryTitle': 419, 'originalTitle': 419, 'isAdult': 4, 'startYear': 4, 'endYear': 4, 'runtimeMinutes': 11, 'genres': 32}

title.episode.tsv
maximums={'tconst': 10, 'parentTconst': 10, 'seasonNumber': 4, 'episodeNumber': 5}

title.principals.tsv
maximums={'tconst': 10, 'ordering': 2, 'nconst': 10, 'category': 19, 'job': 286, 'characters': 463}

title.ratings.tsv
maximums={'tconst': 10, 'averageRating': 4, 'numVotes': 7}




## 3 python, Trouver les différentes valeurs d'un champ
Pour savoir quels sont les différents genres de la table Title_basics, et les différentes professions principales de la table name_basics, pour pouvoir ajouter au set() dans le Datatype dans workbench avec les différentes valeurs

Ces commandes s'exécutent dans le terminal avec python3

In [ ]:
import pandas as pd
df = pd.read_csv("/datalab2020/movies/title.basics.tsv", sep='\t') # lecture du fichier TSV
df['genres'].unique() # donne une liste mélangée, non utilisable telle quelle !
col = df['genres'].replace(',.*', '', regex = True) # récupère les 1er valeurs de genre
col.unique() # les valeurs uniques


import pandas as pd
df = pd.read_csv("/datalab2020/movies/name.basics.tsv", sep='\t') # lecture du fichier TSV
df['primaryProfession'].unique() # donne une liste mélangée, non utilisable telle quelle !
col = df['primaryProfession'].replace(',.*', '', regex = True) # récupère les 1er valeurs de genre
col.unique() # les valeurs uniques

le résultat de cette recherche était:

genres de title.basics.tsv:

'Documentary', 'Animation', 'Comedy', 'Short', 'Romance', 'Action', 'News', 'Drama', 'Fantasy', 'Horror', 'Biography', 'Music', 'Crime', 'Family', 'Adventure', 'History', 'Mystery', 'Musical', 'War', 'Sci-Fi', 'Western', 'Thriller', 'Sport', 'Film-Noir', 'Talk-Show', 'Game-Show', 'Adult', 'Reality-TV'

primaryProfession de name.basics.tsv:

'soundtrack', 'actress', 'actor', 'writer', 'composer', 'music_department', 'director', 'editor', 'cinematographer', 'producer', 'art_director', 'miscellaneous', 'make_up_department', 'assistant_director', 'animation_department', 'camera_department', 'sound_department', 'stunts', 'special_effects', 'production_designer', 'editorial_department', 'production_manager', 'costume_department', 'casting_director', 'costume_designer', 'set_decorator', 'art_department', 'casting_department', 'visual_effects', 'location_management', 'script_department', 'transportation_department', 'manager', 'talent_agent', 'legal', 'publicist', 'executive', 'assistant', 'electrical_department', 'production_department'

## 4 SQL, nouveau tableau avec les différents valeurs d'un champ
Procedure pour créer un nouveau tableau avec les différents genres de title_basics

In [ ]:
CREATE PROCEDURE `updateGenre` (IN genreName VARCHAR(45))
BEGIN
	#Etape 1: vider les cas déjà présents pour ce genre
	DELETE FROM film_genres WHERE genre=genreName;
	# Etape 2: identifier les films du genre doné
	CREATE TEMPORARY TABLE IDS SELECT tconst FROM title_basics WHERE genres LIKE CONCAT('%', 		genreName, '%');
	# Etape 3: on ajoute les lignes dans film_genres
	INSERT INTO film_genres SELECT tconst, genreName FROM IDS;
	#Etape 4: supprimer la table temporaire
	DROP TABLE IDS;
END

## 5 SQL, mettre à jour les fichiers .tsv 

Pour mettre à jour les fichiers .tsv avec les dernières données disponibles sur le site de téléchargement

In [ ]:
# Script de mise à jour
# pour le rendre executable:
# chmod u+x update.sh

for fn in name.basics title.akas title.crew title.basics title.episode title.principals title.ratings
do
    echo $fn
    wget https://datasets.imdbws.com/$fn.tsv.gz --output-document=$fn.tsv.gz
    gunzip --keep --force $fn.tsv.gz
    head -100000 < $fn.tsv > 100k/$fn.100k.tsv
    grep -P "tt[0-9]+6\t" < $fn.tsv > tt6/$fn.tt6.tsv
    
    # Dans sous répertoire 100k, mettre les 100k premières lignes des fichiers
    # Dans sous répertoire tt6, mettre les lignes des fichiers qui contiennent un tconst de la forma "tt.....6"
    
done

## 6 SQL, nettoyage du bdd

Pour éliminer les lignes des différentes tables qui n'ont aucun rapport avec la table principale "title_basics"

In [ ]:
#Par rapoort au modèle, toutes les lignes de title_aka (tconst) doivent correspondre à un film dans #
#title_principals. Identification des tconst correspondants:

CREATE TEMPORARY TABLE IDS       # creation table temporaire
  select A.tconst, A.ordering 
    FROM title_akas AS A 
      LEFT JOIN title_basics AS F 
    ON (A.tconst=F.tconst)
    WHERE F.tconst IS NULL; # récupère les tconst de title_akas qui n'esistent pas dans title_basics

ALTER TABLE IDS ADD INDEX `idx` (tconst ASC); # Ajout d'un index à la table IDS

DELETE FROM title_akas WHERE tconst IN (SELECT tconst FROM IDS); # Suppression des lignes de title_akas *ou* tconst fait parti de la liste dans IDS


#Sans oublier le trigger (table title_akas) qui va bien:
CREATE DEFINER = CURRENT_USER TRIGGER `BDD_EG_movies`.`title_akas_BEFORE_DELETE` BEFORE DELETE ON `title_akas` FOR EACH ROW
BEGIN
    INSERT INTO log (commentaire, tconst, dt) VALUES (CONCAT("Supp: ", OLD.title), OLD.tconst, NOW());
END

## 7 SQL, Backup

Pour faire une sauvegarde de la base de données

In [ ]:
#Pour info, user "reader", MDP "Reader@2020" crée dans mysql. 

mysqldump --no-tablespaces --host=dbxxxxx.hosting-data.io --user=reader --password=Reader@2020 dbXXXXXXX > dbXXXXXXXX.sql 

#Backup mySQL
mysqldump --single-transaction --no-tablespaces -u reader -p grp_movies2 | gzip > grp_movies2.sql.gz

#Pour faire la restauration, cela doit être fait avec un utilisateur qui a l'autorisation d'écriture (cela ne peut pas être fait avec l'utilisateur reader)

Restore :
gunzip < fn.sql.gz | mysql -u user -p DBname

## 8 SQL, Triggers

Pour créer un Trigger qui stocke les informations des modifications apportées dans une table spécifique

Ce trigger se fait dans workbench dans la table où seront effectuées les modifications, cette table doit être liée à la table principale (title_basics dans ce cas) et précédemment la table log doit avoir été créée (où les enregistrements des modifications seront stockés), qui doit également être lié à la table principale.

In [ ]:
CREATE TRIGGER `BDD_EG_movies`.`title_ratings_BEFORE_UPDATE` BEFORE UPDATE ON `title_ratings` FOR EACH ROW
BEGIN
  INSERT INTO log (commentaire, tconst, dt) VALUES (CONCAT("Modification de la notation: ", OLD.averageRating, "->", NEW.averageRating), OLD.tconst, NOW());
END

## 9 python, Modèle alternatif

Modèle Python alternatif qui permet de fermer la connexion avec la base de données en cas de problème

In [ ]:
import pymysql.cursors
import config

# Connect to the database
connection = pymysql.connect(host = '127.0.0.1',
                             user = config.user,
                             password = config.password,
                             db = 'BDD_EG_movies')

# On demanfde à l'utilisateur de donner un nom...
name = input('nom: ') 

try:
    with connection.cursor() as cursor:
        # Assemble la requete SQL qui permet de crée et affecter la variable @name, 
        # celle-ci continet des '%' utilisée pour le LIKE
        sql = "SET @name='%" + name + "%';";
        #~ print(sql) # affiche pour debug
        cursor.execute(sql) # execute le STE @name...
        cursor.execute("SET @N=0;") # execute le STE @name...
        # Execute la recherche elle-même qui utilise @name
        cursor.execute("SELECT @N:=@N+1, nconst, primaryName FROM name_basics WHERE primaryName like @name;")
        # On récupère tout les résultats
        results = cursor.fetchall()
        #print(results)
        # Boucle pour afficher tout les resultats
        for result in results:
            #~ print(result)
            print("{:4d} : {:10s} : {}".format(result[0], result[1], result[2]))
except:
    print("Quelque choses a mal tourné...")
# A la fin, on ferme la connection pour être clean...
finally:
    connection.close()